In [2]:
import pandas as pd
import os

In [3]:
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""

In [4]:
import kaggle


In [5]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('ericpierce/new-york-housing-zillow-api',  force=True, unzip=True)


## Load Dataset

In [119]:
df = pd.read_csv('/content/newyork_housing.csv')

<ipython-input-119-96e460e0107f>:1: DtypeWarning: Columns (2,5,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,305,313,320,32

In [120]:
df.columns

Index(['address/city', 'address/community', 'address/neighborhood',
       'address/state', 'address/streetAddress', 'address/subdivision',
       'address/zipcode', 'bathrooms', 'bedrooms', 'currency',
       ...
       'schools/2/link', 'schools/2/name', 'schools/2/rating',
       'schools/2/size', 'schools/2/studentsPerTeacher',
       'schools/2/totalCount', 'schools/2/type', 'url', 'yearBuilt', 'zpid'],
      dtype='object', length=1507)

In [121]:
df.filter(regex=r'^resoFactsStats/\w+$').columns.tolist()

['resoFactsStats/aboveGradeFinishedArea',
 'resoFactsStats/accessibilityFeatures',
 'resoFactsStats/additionalParcelsDescription',
 'resoFactsStats/appliances',
 'resoFactsStats/architecturalStyle',
 'resoFactsStats/associationAmenities',
 'resoFactsStats/associationFee',
 'resoFactsStats/associationFee2',
 'resoFactsStats/associationFeeIncludes',
 'resoFactsStats/associationName',
 'resoFactsStats/associationName2',
 'resoFactsStats/associationPhone',
 'resoFactsStats/associationPhone2',
 'resoFactsStats/atAGlanceFacts',
 'resoFactsStats/basement',
 'resoFactsStats/bathrooms',
 'resoFactsStats/bathroomsFull',
 'resoFactsStats/bathroomsHalf',
 'resoFactsStats/bathroomsOneQuarter',
 'resoFactsStats/bathroomsPartial',
 'resoFactsStats/bathroomsThreeQuarter',
 'resoFactsStats/bedrooms',
 'resoFactsStats/belowGradeFinishedArea',
 'resoFactsStats/builderModel',
 'resoFactsStats/builderName',
 'resoFactsStats/buildingArea',
 'resoFactsStats/buildingAreaSource',
 'resoFactsStats/buildingFeatu

In [133]:
# filter to columns
columns = ['address/community', 'address/neighborhood', 'address/subdivision', 'schools', 'priceHistory', 'resoFactsStats/yearBuiltEffective', 'resoFactsStats/zoning', 'resoFactsStats/zoningDescription', 'resoFactsStats/view','resoFactsStats/virtualTour','resoFactsStats/waterSources','resoFactsStats/waterfrontFeatures','resoFactsStats/topography','resoFactsStats/utilities','resoFactsStats/vegetation',	'resoFactsStats/windowFeatures','resoFactsStats/woodedArea']
col_to_drop = columns + df.filter(regex='^photos').columns.tolist() + df.filter(regex=r'^priceHistory/\d+/|^schools/\d+/|^resoFactsStats').columns.tolist()
# col_to_drop

In [123]:
df.drop(col_to_drop, axis=1).columns

Index(['address/city', 'address/state', 'address/streetAddress',
       'address/zipcode', 'bathrooms', 'bedrooms', 'currency', 'dateposted',
       'description', 'homeStatus', 'latitude', 'livingArea', 'longitude',
       'price', 'propertyTaxRate', 'url', 'yearBuilt', 'zpid'],
      dtype='object')

In [125]:
df[df.filter(regex=r'^resoFactsStats/\w+$').columns.tolist()[100:140]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75630 entries, 0 to 75629
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   resoFactsStats/otherStructures           0 non-null      float64
 1   resoFactsStats/parcelNumber              67567 non-null  object 
 2   resoFactsStats/parking                   75630 non-null  int64  
 3   resoFactsStats/patioAndPorchFeatures     0 non-null      float64
 4   resoFactsStats/propertyCondition         10914 non-null  object 
 5   resoFactsStats/roofType                  8215 non-null   object 
 6   resoFactsStats/securityFeatures          0 non-null      float64
 7   resoFactsStats/sewer                     0 non-null      float64
 8   resoFactsStats/spaFeatures               0 non-null      float64
 9   resoFactsStats/specialListingConditions  1044 non-null   object 
 10  resoFactsStats/stories                   47813

## curate columns to consider

In [126]:
columns_to_keep = ['address/city', 'address/state', 'address/streetAddress',
       'address/zipcode', 'bathrooms', 'bedrooms', 'dateposted',
       'description', 'homeStatus', 'latitude', 'livingArea', 'longitude',
       'price', 'propertyTaxRate', 'url', 'yearBuilt', 'zpid', 'resoFactsStats/furnished', 'resoFactsStats/cityRegion', 'resoFactsStats/garageSpaces', 'resoFactsStats/hasCooling', 'resoFactsStats/hasHeating', 'resoFactsStats/hasOpenParking', 'resoFactsStats/hasPetsAllowed', 'resoFactsStats/homeType', 'resoFactsStats/parking', 'resoFactsStats/yearBuilt',
        'photos/0','photos/1','photos/2','photos/3','photos/4','photos/5','photos/6','photos/7','photos/8','photos/9']

df = df[columns_to_keep].dropna()
df['dateposted'] = pd.to_datetime(df[columns_to_keep].dropna()['dateposted'].astype(int), unit='ms')

In [131]:
df.to_csv("newyork_housing_formatted.csv", index=False)

In [132]:
df

,address/city,address/state,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,...,photos/0,photos/1,photos/2,photos/3,photos/4,photos/5,photos/6,photos/7,photos/8,photos/9
0,New York,NY,60 Terrace View Ave,10463.0,2.0,5.0,2021-01-08 19:33:26,"Discover Marble Hill, a neighborhood rich with...",FOR_SALE,40.877743,...,https://photos.zillowstatic.com/fp/006eb7ab9c3...,https://photos.zillowstatic.com/fp/8b10648864c...,https://photos.zillowstatic.com/fp/08edf887608...,https://photos.zillowstatic.com/fp/d61f7b6017e...,https://photos.zillowstatic.com/fp/4a8d68dccb2...,https://photos.zillowstatic.com/fp/9fd2c273f39...,https://photos.zillowstatic.com/fp/7bc3ee9bc4c...,https://photos.zillowstatic.com/fp/644388de162...,https://photos.zillowstatic.com/fp/7bc3ee9bc4c...,https://photos.zillowstatic.com/fp/ca5af4b6f89...
1,Bronx,NY,625 W 246th St,10471.0,8.0,8.0,2020-07-28 20:23:00,EXCLUSIVE BRAND NEW\nLavish Newly Built 8-Bd. ...,FOR_SALE,40.892689,...,https://photos.zillowstatic.com/fp/9c91425f8fb...,https://photos.zillowstatic.com/fp/82a019ae300...,https://photos.zillowstatic.com/fp/d1d0ed56f85...,https://photos.zillowstatic.com/fp/65103165fc7...,https://photos.zillowstatic.com/fp/8d20b0ec4ef...,https://photos.zillowstatic.com/fp/7b9f904cb69...,https://photos.zillowstatic.com/fp/20e8df506bb...,https://photos.zillowstatic.com/fp/f356dcac67b...,https://photos.zillowstatic.com/fp/8d4738c7035...,https://photos.zillowstatic.com/fp/381211d9a4b...
2,Bronx,NY,716 W 231st St,10463.0,3.0,4.0,2020-06-20 15:51:00,This 4233 square foot single family home has 4...,FOR_SALE,40.883419,...,https://photos.zillowstatic.com/fp/797eb4a9695...,https://photos.zillowstatic.com/fp/956034ac704...,https://photos.zillowstatic.com/fp/b799eae9027...,https://photos.zillowstatic.com/fp/4b393c3aff3...,https://photos.zillowstatic.com/fp/a207d715629...,https://photos.zillowstatic.com/fp/5724a2d40d9...,https://photos.zillowstatic.com/fp/e4f91d8b114...,https://photos.zillowstatic.com/fp/6a4794be3fa...,https://photos.zillowstatic.com/fp/8f8ba2f927e...,https://photos.zillowstatic.com/fp/b94ce3a3760...
3,Bronx,NY,750 W 232nd St,10463.0,6.0,5.0,2020-09-22 22:25:00,EXCLUSIVE NEW TO MARKET\nPrime Renovation Oppo...,FOR_SALE,40.885033,...,https://photos.zillowstatic.com/fp/36a83a8d300...,https://photos.zillowstatic.com/fp/90000a23b0b...,https://photos.zillowstatic.com/fp/9d7bb173624...,https://photos.zillowstatic.com/fp/6e171567e67...,https://photos.zillowstatic.com/fp/f75d80b80be...,https://photos.zillowstatic.com/fp/8c2d97e9c45...,https://photos.zillowstatic.com/fp/ba73c3d72d9...,https://photos.zillowstatic.com/fp/ae9d8311876...,https://photos.zillowstatic.com/fp/adb5f981b0b...,https://photos.zillowstatic.com/fp/a460a6dfd4f...
6,Bronx,NY,1 Ploughmans Bush,10471.0,3.0,3.0,2020-11-23 18:21:15,Looking for a country retreat not too far from...,FOR_SALE,40.892391,...,https://photos.zillowstatic.com/fp/ad7898dcfc2...,https://photos.zillowstatic.com/fp/721f142ae3e...,https://photos.zillowstatic.com/fp/ba9de61c6b5...,https://photos.zillowstatic.com/fp/fa146ed0660...,https://photos.zillowstatic.com/fp/977143a1ea9...,https://photos.zillowstatic.com/fp/e34292142d1...,https://photos.zillowstatic.com/fp/7c91b025eff...,https://photos.zillowstatic.com/fp/74db6a0d59a...,https://photos.zillowstatic.com/fp/4bf4181fe4d...,https://photos.zillowstatic.com/fp/9f0346fbe97...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74742,Forest Hills,NY,110-20 71st Rd APT 420,11375.0,1.0,2.0,2020-11-30 19:44:00,Beautifully renovated 2 Bedroom/TRUE JR4 has a...,FOR_SALE,40.721622,...,https://photos.zillowstatic.com/fp/f051b53f67b...,https://photos.zillowstatic.com/fp/532b5ac250c...,https://photos.zillowstatic.com/fp/450e5b462f7...,https://photos.zillowstatic.com/fp/ed91bcaf7bb...,https://photos.zillowstatic.com/fp/c74c68b5755...,https://photos.zillowstatic.com/fp/647c18ac015...,https://photos.zillowstatic.com/fp/19f32838c98..